In [1]:
# !pip3 install PyPDF2

In [2]:
from PyPDF2 import PdfFileReader, PdfFileWriter

def pdf_cat(input_files, output_stream):
    input_streams = []
    try:
        # First open all the files, then produce the output file, and
        # finally close the input files. This is necessary because
        # the data isn't read from the input files until the write
        # operation. Thanks to
        # https://stackoverflow.com/questions/6773631/problem-with-closing-python-pypdf-writing-getting-a-valueerror-i-o-operation/6773733#6773733
        for input_file in input_files:
            input_streams.append(open(input_file, 'rb'))
        writer = PdfFileWriter()
        for reader in map(PdfFileReader, input_streams):
            for n in range(reader.getNumPages()):
                writer.addPage(reader.getPage(n))
        output_stream = open(output_stream, 'wb')
        writer.write(output_stream)
    finally:
        for f in input_streams:
            f.close()
        output_stream.close()


In [3]:
import datetime, os

today = datetime.date.today() + datetime.timedelta(days=1) # +1 day
path = './pdfs/'

region_zones = {
    'Республика Хакасия' : 2,
    'Алтайский край' : 2,
    'Красноярский край' : 2,
    'Кемеровская область' : 2,    
    'Забайкальский край' : 2,
    'Новосибирская область' : 2,
    'Республика Тыва' : 2,
    'Республика Бурятия' : 2,
    'Волгоградская область' : 1,
    'Краснодарский край' : 1,    
    'Ленинградская область' : 1,
    'Мурманская область' : 1,    
    'Пермский край' : 1,
    'Свердловская область' : 1,
    'Ставропольский край' : 1,
    'Тульская область' : 1,
    'Хабаровский край' : 0,
    'Приморский край' : 0,
   }

files = []
file_error = 0
for ter in region_zones:
    full_path = path + today.strftime("%Y") + '/' + today.strftime("%m") + '/' + today.strftime("%d") + '/' + 'Контрольные часы ' + ter + ' на ' + today.strftime("%d-%m-%Y") + '.pdf'
    if os.path.exists(full_path):
        files += [full_path]
    else:
        file_error += 1
files

['./pdfs/2021/11/16/Контрольные часы Республика Хакасия на 16-11-2021.pdf',
 './pdfs/2021/11/16/Контрольные часы Алтайский край на 16-11-2021.pdf',
 './pdfs/2021/11/16/Контрольные часы Красноярский край на 16-11-2021.pdf',
 './pdfs/2021/11/16/Контрольные часы Кемеровская область на 16-11-2021.pdf',
 './pdfs/2021/11/16/Контрольные часы Забайкальский край на 16-11-2021.pdf',
 './pdfs/2021/11/16/Контрольные часы Новосибирская область на 16-11-2021.pdf',
 './pdfs/2021/11/16/Контрольные часы Республика Тыва на 16-11-2021.pdf',
 './pdfs/2021/11/16/Контрольные часы Республика Бурятия на 16-11-2021.pdf',
 './pdfs/2021/11/16/Контрольные часы Волгоградская область на 16-11-2021.pdf',
 './pdfs/2021/11/16/Контрольные часы Краснодарский край на 16-11-2021.pdf',
 './pdfs/2021/11/16/Контрольные часы Ленинградская область на 16-11-2021.pdf',
 './pdfs/2021/11/16/Контрольные часы Мурманская область на 16-11-2021.pdf',
 './pdfs/2021/11/16/Контрольные часы Пермский край на 16-11-2021.pdf',
 './pdfs/2021/1

In [4]:
pdf_cat(files, path + today.strftime("%Y") + '/' + today.strftime("%m") + '/' + today.strftime("%d") + '/' + 'Контрольные часы на ' + today.strftime("%d-%m-%Y") + '.pdf')

In [5]:
import smtplib                                              # Импортируем библиотеку по работе с SMTP
import ssl                                                  # Библиотека для работы с SSL
import os                                                   # Функции для работы с операционной системой, не зависящие от используемой операционной системы
 
# Добавляем необходимые подклассы - MIME-типы
import mimetypes                                            # Импорт класса для обработки неизвестных MIME-типов, базирующихся на расширении файла
from email import encoders                                  # Импортируем энкодер
from email.mime.base import MIMEBase                        # Общий тип
from email.mime.text import MIMEText                        # Текст/HTML
from email.mime.image import MIMEImage                      # Изображения
from email.mime.audio import MIMEAudio                      # Аудио
from email.mime.multipart import MIMEMultipart              # Многокомпонентный объект
 

def send_email(addr_to, msg_subj, msg_text, files):
    addr_from = "omima@suek.ru"                             # Отправитель
    password  = r'VxjNOJV]!3'                               # Пароль
 
    msg = MIMEMultipart()                                   # Создаем сообщение
    msg['From']    = addr_from                              # Адресат
    msg['To']      = addr_to                                # Получатель
    msg['Subject'] = msg_subj                               # Тема сообщения
 
    body = msg_text                                         # Текст сообщения
    msg.attach(MIMEText(body, 'plain'))                     # Добавляем в сообщение текст
 
    process_attachement(msg, files)
 
    #==========================================================================================================================
    # Create a secure SSL context
    context = ssl.SSLContext(ssl.PROTOCOL_TLSv1_2)
    # Try to log in to server and send email
    try:
        server = smtplib.SMTP("kmr-relay.suek.ru", 587)      # Создаем объект SMTP
        server.ehlo()                                           # Can be omitted
        server.starttls(context=context)                        # Начинаем шифрованный обмен по TLS
        #server.set_debuglevel(True)                            # Включаем режим отладки, если не нужен - можно закомментировать
        server.ehlo()                                           # Can be omitted
        server.login(addr_from, password)                       # Получаем доступ
        server.send_message(msg)                                # Отправляем сообщение
        server.quit()                                           # Выходим
    except Exception as e:
        # Print any error messages to stdout
        print('Возникла проблема при отправке. Ошибка: ' + str(e))        
    #==========================================================================================================================

def process_attachement(msg, files):                        # Функция по обработке списка, добавляемых к сообщению файлов
    for f in files:
        if os.path.isfile(f):                               # Если файл существует
            attach_file(msg,f)                              # Добавляем файл к сообщению
        elif os.path.exists(f):                             # Если путь не файл и существует, значит - папка
            dir = os.listdir(f)                             # Получаем список файлов в папке
            for file in dir:                                # Перебираем все файлы и...
                attach_file(msg,f+"/"+file)                 # ...добавляем каждый файл к сообщению

def attach_file(msg, filepath):                             # Функция по добавлению конкретного файла к сообщению
    filename = os.path.basename(filepath)                   # Получаем только имя файла
    ctype, encoding = mimetypes.guess_type(filepath)        # Определяем тип файла на основе его расширения
    if ctype is None or encoding is not None:               # Если тип файла не определяется
        ctype = 'application/octet-stream'                  # Будем использовать общий тип
    maintype, subtype = ctype.split('/', 1)                 # Получаем тип и подтип
    if maintype == 'text':                                  # Если текстовый файл
        with open(filepath) as fp:                          # Открываем файл для чтения
            file = MIMEText(fp.read(), _subtype=subtype)    # Используем тип MIMEText
            fp.close()                                      # После использования файл обязательно нужно закрыть
    elif maintype == 'image':                               # Если изображение
        with open(filepath, 'rb') as fp:
            file = MIMEImage(fp.read(), _subtype=subtype)
            fp.close()
    elif maintype == 'audio':                               # Если аудио
        with open(filepath, 'rb') as fp:
            file = MIMEAudio(fp.read(), _subtype=subtype)
            fp.close()
    else:                                                   # Неизвестный тип файла
        with open(filepath, 'rb') as fp:
            file = MIMEBase(maintype, subtype)              # Используем общий MIME-тип
            file.set_payload(fp.read())                     # Добавляем содержимое общего типа (полезную нагрузку)
            fp.close()
            encoders.encode_base64(file)                    # Содержимое должно кодироваться как Base64
    file.add_header('Content-Disposition', 'attachment', filename=filename) # Добавляем заголовки
    msg.attach(file)                                        # Присоединяем файл к сообщению


files = [path + today.strftime("%Y") + '/' + today.strftime("%m") + '/' + today.strftime("%d") + '/' + 'Контрольные часы на ' + today.strftime("%d-%m-%Y") + '.pdf', # Список файлов, если вложений нет, то files=[]
        # "file2_path",                                      
        # "dir_path"  # Если нужно отправить все файлы из заданной папки, нужно указать её
        ]       

# text_mail = ter + "\n\nКонтрольные часы на " + today.strftime("%d.%m.%Y") + " (МСК" + hour_pref + ")\n\n" + "Это сообщение сформировано автоматически, отвечать на него не нужно."

text_mail = "Прогноз контрольных часов (часы максимума, СКЧ) на " + today.strftime("%d.%m.%Y")+ ".\n"
# + " по регионам:\n"
# for ter in region_zones:
#     if region_zones[ter] == 0:
#         text_mail += " - " + ter + " (время хабаровское).\n"
#     else:
#         text_mail += " - " + ter + " (время московское).\n"
text_mail += "В указанные часы рекомендуется принимать меры по снижению потребления электроэнергии.\n\n"
text_mail += "Это сообщение сформировано автоматически, отвечать на него не нужно.\n\n"
text_mail += "При необходимости корректировки списка рассылки обращаться к Богданову А.В. по Email: BogdanovAndV@sibgenco.ru .\n\n"
text_mail += "Это сообщение и все приложения к нему строго конфиденциальны и предназначены исключительно для использования получателем(адресатом), которому они адресованы. Использование информации, содержащейся в настоящем сообщении, допускается с соблюдением требований законодательства РФ об охране конфиденциальной информации и соглашений о коммерческой тайне."


In [6]:
import pandas as pd

pathP='/mnt/Production/СМП'

In [7]:
addr_1cz = pd.read_excel(pathP + '/СКЧ списки на отправку.xlsx', sheet_name='1цз')
# addr_1cz

In [8]:
addr_1cz.columns

Index(['Ленинградская область', 'Волгоградская область', 'Мурманская область',
       'Тульская область', 'Ставропольский край', 'Пермский край',
       'Краснодарский край', 'Свердловская область'],
      dtype='object')

In [9]:
addr_2cz = pd.read_excel(pathP + '/СКЧ списки на отправку.xlsx', sheet_name='2цз')
# addr_2cz

In [10]:
addr_2cz.columns

Index(['Алтайский край', 'Красноярский край', 'Кемеровская область',
       'Республика Хакасия', 'Новосибирская область', 'Забайкальский край',
       'Республика Тыва', 'Республика Бурятия'],
      dtype='object')

In [11]:
addr_Ncz = pd.read_excel(pathP + '/СКЧ списки на отправку.xlsx', sheet_name='нцз')
# addr_Ncz

In [12]:
addr_Ncz.columns

Index(['Хабаровский край', 'Приморский край'], dtype='object')

In [13]:
addr_all = pd.read_excel(pathP + '/СКЧ списки на отправку.xlsx', sheet_name='Все регионы')
# addr_all

In [14]:
addr_cz = pd.concat([addr_1cz, addr_2cz, addr_Ncz], axis=1)
# addr_cz

In [15]:
addr_dict = {}

for reg in region_zones:
    addr_dict[reg] = []
    for value in addr_cz[reg].dropna().values:
#         print(value.split()[-1:][0])
        addr_dict[reg].append(value.split()[-1:][0])
    addr_dict[reg] = ", ".join(addr_dict[reg])
# addr_dict

In [16]:
list_all = []
for value in addr_all['Все регионы'].dropna().values:
    print(value.split()[-1:][0])
    list_all.append(value.split()[-1:][0])
list_all = ",".join(list_all)
list_all

VelichkoYA@sibgenco.ru
SaharovSN@suek.ru
Sitovatv@sibgenco.ru
TretiakovaOlS@sibgenco.ru
kazarinms@sibgenco.ru


'VelichkoYA@sibgenco.ru,SaharovSN@suek.ru,Sitovatv@sibgenco.ru,TretiakovaOlS@sibgenco.ru,kazarinms@sibgenco.ru'

In [17]:
# Получатель
# addr_to   = 'KazarinMS@sibgenco.ru, VelichkoYA@sibgenco.ru'
addr_to = list_all

In [18]:
print('emails =', addr_to)

emails = VelichkoYA@sibgenco.ru,SaharovSN@suek.ru,Sitovatv@sibgenco.ru,TretiakovaOlS@sibgenco.ru,kazarinms@sibgenco.ru


In [19]:
# Использование функции send_email
send_email(addr_to, "Контрольные часы на " + today.strftime("%d.%m.%Y"), text_mail, files)
print('sended')

sended
